<a href="https://colab.research.google.com/github/SiliconJackets/sscs-ose-code-a-chip.github.io/blob/main/VLSI24/submitted_notebooks/SJSystolicArray/SystolicArray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Systolic Array With Openlane

```
Copyright 2023 SiliconJackets
SPDX-License-Identifier: GPL-3.0-or-later
```

Running a 3x3 systolic array design inspired by [EYERISS](https://courses.cs.washington.edu/courses/cse550/21au/papers/CSE550.Eyeriss.pdf) design thru the [OpenLane](https://github.com/The-OpenROAD-Project/OpenLane/) GDS to RTL flow targeting the [open source SKY130 PDK](https://github.com/google/skywater-pdk/) with the addition of dual-port RAM macros generated by [OpenRAM](https://github.com/VLSIDA/OpenRAM).

|Name|Affiliation| Email |IEEE Member|SSCS Member|
|:--:|:----------:|:----------:|:----------:|:----------:|
|Zachary Ellis|Georgia Institute of Technology|zellis7@gatech.edu|Yes|Yes|
|Nealson Li|Georgia Institute of Technology|nealson@gatech.edu|Yes|Yes|
|Addison Elliott|Georgia Institute of Technology|addisonelliott@gatech.edu|Yes|Yes|
|Zeyan Wu|Georgia Institute of Technology|zwu477@gatech.edu|Yes|Yes|

In [3]:
#@title Install Dependencies {display-mode: "form"}
#@markdown Click the ▷ button to setup the digital design environment based on [conda-eda](https://github.com/hdl/conda-eda).

#@markdown Main components we will install

#@markdown *   Open_pdks.sky130a : a PDK installer for open-source EDA tools.
#@markdown *   Openlane : an automated RTL to GDSII flow based on several components including OpenROAD, Yosys, Magic, Netgen, CVC, SPEF-Extractor, KLayout and a number of custom scripts for design exploration and optimization.
#@markdown *   GDSTK : a C++ library for creation and manipulation of GDSII and OASIS files.

import os
import pathlib
import sys

!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba
conda_prefix_path = pathlib.Path('conda-env')
site_package_path = conda_prefix_path / 'lib/python3.7/site-packages'
sys.path.append(str(site_package_path.resolve()))

CONDA_PREFIX = str(conda_prefix_path.resolve())
PATH = os.environ['PATH']
#LD_LIBRARY_PATH = os.environ.get('LD_LIBRARY_PATH', '')

%env CONDA_PREFIX={CONDA_PREFIX}
%env PATH={CONDA_PREFIX}/bin:{PATH}
%env LD_LIBRARY_PATH={CONDA_PREFIX}/lib:{LD_LIBRARY_PATH}

!bin/micromamba create --yes --prefix $CONDA_PREFIX
!echo 'python ==3.7*' >> {CONDA_PREFIX}/conda-meta/pinned

!bin/micromamba install --quiet \
                        --yes \
                        --prefix $CONDA_PREFIX \
                        --channel litex-hub \
                        --channel main \
                        open_pdks.sky130a \
                        openlane

!bin/micromamba install --quiet \
                        --yes \
                        --prefix $CONDA_PREFIX \
                        --channel conda-forge \
                        gdstk

!python -m pip install libparse libparse
PATH = os.environ['PATH']
%env CONDA_PREFIX={CONDA_PREFIX}
%env PATH={CONDA_PREFIX}/bin:{PATH}



bin/micromamba
env: CONDA_PREFIX=/tmp/OpenRAM/conda-env
env: PATH=/tmp/OpenRAM/conda-env/bin:/content/conda-env/bin:/content/conda-env/bin:/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
env: LD_LIBRARY_PATH={CONDA_PREFIX}/lib:{LD_LIBRARY_PATH}
Empty environment created at prefix: /tmp/OpenRAM/conda-env
warning  libmamba [binutils_impl_linux-64-2.38-h2a08ee3_1] The following files were already present in the environment:
    - lib/liblsan.so
    - lib/liblsan.so.0
    - lib/liblsan.so.0.0.0
    - share/info/libgomp.info
    - share/info/libquadmath.info
    - share/licenses/gcc-libs/RUNTIME.LIBRARY.EXCEPTION
    - share/licenses/gcc-libs/RUNTIME.LIBRARY.EXCEPTION.gomp_copy
    - share/licenses/libstdc++/RUNTIME.LIBRARY.EXCEPTION
warning  libmamba [libblas-3.9.0-16_linux64_openblas] The following files were already present in the environment:
    - lib/libblas.so
warning  libmamba

In [4]:
%cd /tmp
! git clone -b v1.2.8 https://github.com/VLSIDA/OpenRAM > /dev/null 2>&1

/tmp


In [5]:
%cd /tmp/OpenRAM
! ./install_conda.sh > /dev/null 2>&1

/tmp/OpenRAM


In [6]:
! python -m pip install -r requirements.txt > /dev/null 2>&1
%env OPENRAM_ROOT=/tmp/OpenRAM
%env OPENRAM_HOME="/tmp/OpenRAM/compiler"
%env OPENRAM_TECH="/tmp/OpenRAM/technology"
%env PYTHONPATH=$OPENRAM_HOME
%env PYTHONPATH="$OPENRAM_HOME:$OPENRAM_TECH/sky130:$OPENRAM_TECH/sky130/custom"

env: OPENRAM_ROOT=/tmp/OpenRAM
env: OPENRAM_HOME="/tmp/OpenRAM/compiler"
env: OPENRAM_TECH="/tmp/OpenRAM/technology"
env: PYTHONPATH=$OPENRAM_HOME
env: PYTHONPATH="$OPENRAM_HOME:$OPENRAM_TECH/sky130:$OPENRAM_TECH/sky130/custom"


In [7]:
! source miniconda/bin/activate && make pdk > /dev/null 2>&1

In [8]:
! make install > /dev/null 2>&1

In [9]:
%cd /tmp/OpenRam

[Errno 2] No such file or directory: '/tmp/OpenRam'
/tmp/OpenRAM


In [10]:
%%writefile macros/sram_configs/myconfig.py

"""
Dual port (1 read/write + 1 read only) 1 kbytes SRAM with byte write.
"""
word_size = 8

num_words = 16

human_byte_size = "{:.0f}kbytes".format((word_size * num_words)/1024/8)

# Allow byte writes
write_size = 2 # Bits

# Dual port
num_rw_ports = 1
num_r_ports = 1
num_w_ports = 0
ports_human = '1rw1r'

import os
exec(open(os.path.join(os.path.dirname(__file__), 'sky130_sram_common.py')).read())

Writing macros/sram_configs/myconfig.py


In [16]:

config_file = "macros/sram_configs/myconfig.py"


#@markdown ---
#@markdown ### Number of bits for each memory word:
word_size = 8 #@param {type:"raw"}

#@markdown ---
#@markdown ### Total number of memory words:
num_words = 64 #@param {type:"raw"}

#@markdown ---
#@markdown ### Lowest number of writable bits in one write cylce:
write_size = 8 #@param {type:"raw"}

#@markdown ---
#@markdown ### Number of Read/Write ports:
num_rw_ports = 2 #@param {type:"raw"}

#@markdown ---
#@markdown ### Number of Read only ports:
num_r_ports = 0 #@param {type:"raw"}

#@markdown ---
#@markdown ### Number of Write only ports:
num_w_ports = 0 #@param {type:"raw"}

#@markdown ---



! sed -i "s/^word_size.*$/word_size = $word_size/" $config_file
! sed -i "s/^num_words.*$/num_words = $num_words/" $config_file
! sed -i "s/^write_size.*$/write_size = $write_size/" $config_file
! sed -i "s/^num_rw_ports.*$/num_rw_ports = $num_rw_ports/" $config_file
! sed -i "s/^num_r_ports.*$/num_r_ports = $num_r_ports/" $config_file
! sed -i "s/^num_w_ports.*$/num_w_ports = $num_w_ports/" $config_file
! sed -i "s/^FIXME.*$//" $config_file

In [17]:
! cd macros/ && make myconfig.ok

Building myconfig
[openram.globals/init_openram]: Initializing OpenRAM...
[openram.globals/setup_paths]: OpenRAM source code found in /tmp/OpenRAM/compiler
[openram.globals/setup_paths]: Temporary files saved in /results/myconfig/tmp/
[openram.globals/read_config]: Configuration file is /tmp/OpenRAM/macros/sram_configs/myconfig.py
[openram.globals/read_config]: Output saved in /tmp/OpenRAM/macros/myconfig/
[openram.globals/install_conda]: Creating conda setup...
[openram.globals/import_tech]: Tech directory found in /tmp/OpenRAM/technology:/tmp/OpenRAM/compiler/../technology
[openram.globals/import_tech]: Adding technology path: /tmp/OpenRAM/technology
[openram.globals/import_tech]: Adding technology path: /tmp/OpenRAM/compiler/../technology
[openram.globals/purge_temp]: Purging temp directory: /results/myconfig/tmp/
[openram.globals/setup_bitcell]: Using bitcell: bitcell_2port
[openram.characterizer/<module>]: Initializing characterizer...
[openram.characterizer/<module>]: Analytical 